## Model

In [ ]:
pwd

In [ ]:
import gc
gc.collect()

import numpy as np
import pandas as pd
import os

fp = "D:\\DBC\\Dataset\\images\\Images"
print(os.listdir(fp))
dog_classes = os.listdir(fp)

In [ ]:
breeds = [breed.split('-',1)[1] for breed in dog_classes]
print(breeds)

In [ ]:
from itertools import chain
x = []
y = []

fullpaths = ["D:\\DBC\\Dataset\\images\\Images\\{}".format(dog_class) for dog_class in dog_classes]

for counter, fullpath in enumerate(fullpaths):
    for imgname in os.listdir(fullpath):
        x.append([fullpath + '\\' + imgname])
        y.append(breeds[counter])
        
print(x[:10],'\n')
print(y[:10],'\n')

x = list(chain.from_iterable(x))
print(x[:10],'\n')
len(x)

In [ ]:
import random

combined = list(zip(x,y))
print(combined[:10],'\n')
random.shuffle(combined)
print(combined[:10],'\n')
x[:],y[:] = zip(*combined)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

from matplotlib.image import imread

plt.figure(figsize=(18,18))

for counter, i in enumerate(random.sample(range(0, len(x)),9)):
    plt.subplot(3,3 ,counter + 1)
    plt.subplots_adjust(hspace=0.3)
    filename = x[i]
    image = imread(filename)
    plt.imshow(image)
    plt.title(y[i],fontsize=12)
    
plt.show()

In [ ]:
x = x[:2300]
y = y[:2300]

In [ ]:
import pickle
pickle.dump(y,open('y.dat','wb'))

In [ ]:
y = pickle.load(open('y.dat','rb'))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
le.fit(y)
y_ohe = to_categorical(le.transform(y),len(breeds))
print(y_ohe.shape)

y_ohe = np.array(y_ohe)

In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array,load_img

img_data = np.array([img_to_array(load_img(img, target_size=(299,299))) for img in x])
print(img_data.shape)

x_train,x_test,y_train,y_test = train_test_split(img_data,y_ohe,test_size = 0.2,random_state = 2)
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size = 0.2,random_state = 2)

print("Training dataset size: ",x_train.shape)
print("Validation dataset size: ",x_val.shape)
print("Testing dataset size: ",x_test.shape)
print("Training label size: ",y_train.shape)
print("Validation label size: ",y_val.shape)
print("Testing label size: ",y_test.shape)

import gc
del img_data
gc.collect()

In [ ]:
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  rotation_range=30,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True)

train_generator = train_datagen.flow(x_train,y_train,shuffle=False,batch_size = batch_size,seed =1)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow(x_val,y_val,shuffle=False,batch_size = batch_size,seed =1)

In [ ]:
img_id = 2
dog_generator = train_datagen.flow(x_train[img_id:img_id+1],y_train[img_id:img_id+1]
                                     ,shuffle=False,batch_size = batch_size,seed =1)

plt.figure(figsize=(20,20))
dogs = [next(dog_generator) for i in range(0,5)]
for counter, dog in enumerate(dogs):
    plt.subplot(1,5,counter+1)
    plt.imshow(dog[0][0])
    
plt.show()

In [ ]:
from keras import models
from keras import layers
from keras.optimizers import adam_v2
from keras.layers import GlobalAveragePooling2D,Dense,Flatten,Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

base_model = InceptionV3(weights='imagenet',include_top=False,input_shape=(299,299,3))

model=models.Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(512,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(len(breeds),activation='softmax'))

print("Number of trainable weights before freezing the base layer: ",len(model.trainable_weights))
model.layers[0].trainable = False
print("Number of trainable weights after freezing the base layer: ",len(model.trainable_weights))

In [ ]:
o = adam_v2.Adam(learning_rate=0.0001)
model.compile(o,loss = 'categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
train_steps_per_epoch = x_train.shape[0] // batch_size
val_steps_per_epoch = x_val.shape[0] // batch_size

epochs = 20

history = model.fit_generator(train_generator,steps_per_epoch=train_steps_per_epoch,validation_data=val_generator,
                             validation_steps=val_steps_per_epoch,epochs = epochs,verbose = 1)

In [ ]:
f, (ax1,ax2) = plt.subplots(1, 2,figsize=(12,4))
t = f.suptitle('Transfer Learning Performance',fontsize = 12)
f.subplots_adjust(top=0.85,wspace=0.3)

epoch_list = list(range(1,epochs+1))
ax1.plot(epoch_list,history.history['accuracy'],label='Train Accuracy')
ax1.plot(epoch_list,history.history['val_accuracy'],label='Validation Accuracy')
ax1.set_xticks(np.arange(0,epochs+1,5))
ax1.set_ylabel('Accuracy value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc = 'best')

ax2.plot(epoch_list,history.history['loss'],label='Train Loss')
ax2.plot(epoch_list,history.history['val_loss'],label='Validation Loss')
ax2.set_xticks(np.arange(0,epochs+1,5))
ax2.set_ylabel('Loss value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc = 'best')

In [ ]:
x_test1 = x_test/255.
test_predictions = model.predict(x_test1)

predictions = le.classes_[np.argmax(test_predictions,axis=1)]
target_labels = le.classes_[np.argmax(y_test,axis=1)]

predict_df = pd.DataFrame({'Target_Labels':target_labels, 'Predictions':predictions})
predict_df.head(20)

In [ ]:
correct = (target_labels == predictions)
accuracy = correct.sum() / correct.size
print(accuracy)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(target_labels,predictions))

## Particular Image

In [ ]:
model.save('final_model.h5')

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('final_model.h5')

In [ ]:
x_test1 = x_test/255.
test_predictions = model.predict(x_test1)

predictions = le.classes_[np.argmax(test_predictions,axis=1)]
target_labels = le.classes_[np.argmax(y_test,axis=1)]

predict_df = pd.DataFrame({'Target_Labels':target_labels, 'Predictions':predictions})
predict_df.head(20)

In [ ]:
correct = (target_labels == predictions)
accuracy = correct.sum() / correct.size
print(accuracy)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(target_labels,predictions))

In [ ]:
def load_image(filename):
    img = load_img(filename, color_mode = "rgb", target_size=(299, 299, 3))
    img = img_to_array(img)
    img = img.reshape(1, 299, 299, 3)
    img = img.astype('float32')
    img = img / 255.0
    return img

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

def output(image):
    img = load_image(image)
    test_predictions = model.predict(img)
    print(test_predictions)
    predictions = le.classes_[np.argmax(test_predictions,axis=1)]
    print(predictions[0])

In [ ]:
from skimage import io
 
img = io.imread("Shih Tzu.jpg")
io.imshow(img)

In [ ]:
output('Shih Tzu.jpg')

In [ ]:
img = io.imread("American_Staffordshire_terrier.jpg")
io.imshow(img)

In [ ]:
output('American_Staffordshire_terrier.jpg')

In [ ]:
img = io.imread("labrador retriever.jpg")
io.imshow(img)

In [ ]:
output("labrador retriever.jpg")

## Web Page

In [ ]:
from __future__ import division, print_function
import sys
import os
import glob
import re
import numpy as np
import pandas as pd
import random
import pickle
from sklearn.preprocessing import LabelEncoder

from itertools import chain
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from keras.models import load_model

import gc
gc.collect()

y = pickle.load(open('y.dat','rb'))

le = LabelEncoder()
le.fit(y)


app = Flask(__name__)
UPLOAD_FOLDER = './uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


MODEL_PATH ='final_model.h5'


model = load_model(MODEL_PATH)


def model_predict(img_path, model):
    
    img = load_img(img_path, color_mode = "rgb", target_size=(299, 299, 3))
    x = img_to_array(img)
    x = x/255.0
    x = np.expand_dims(x, axis=0)
    test_predictions = model.predict(x)
    predictions = le.classes_[np.argmax(test_predictions,axis=1)]
    return predictions[0]


@app.route("/", methods=['GET', 'POST'])
def home():
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        if 'file1' not in request.files:
            return 'there is no file1 in form!'
        file1 = request.files['file1']
        path = os.path.join(app.config['UPLOAD_FOLDER'], file1.filename)
        file1.save(path)
        return model_predict(path,model)
    
    return None



if __name__ == '__main__':
    app.run(debug=False)